In [1]:
import os
%pwd

'c:\\Users\\rohit\\OneDrive\\Desktop\\flask_projs\\Text_summarizer_end_to_end\\notebooks'

In [2]:
os.chdir("../")

In [16]:
%pwd

'c:\\Users\\rohit\\OneDrive\\Desktop\\flask_projs\\Text_summarizer_end_to_end'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir : Path
    data_path : Path
    model_path : Path
    tokenizer_path : Path
    metric_file_name : Path 

In [18]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
        )

        return model_evaluation_config

In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm
import evaluate

In [23]:
class ModelEvaluation:
    def __init__(self,config : ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self,list_of_elements,batch_size):
        for i in range(0,len(list_of_elements),batch_size):
            yield list_of_elements[i:i+batch_size]
    
    def calculate_metric_on_test_ds(self,dataset,metric,model,tokenizer,
                               batch_size=16,device="cuda",
                               column_text="article",
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text],batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary],batch_size))

        for article_batch, target_batch in tqdm(zip(article_batches,target_batches)):
            inputs = tokenizer(article_batch,max_length=1024,truncation=True,  #generates token for each word
                            padding="max_length",return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                    attention_mask=inputs["attention_mask"].to(device),  # genrated word tokens from decoder side 
                                    length_penalty=0.8,num_beams=8,max_length=128)

            decoded_summaries = [tokenizer.decode(s,skip_special_tokens=True,           # converts tokens to words
                                                clean_up_tokenization_spaces=True)
                                for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries,references=target_batch)
        score = metric.compute()
        return score
    

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        data_samsum_pt = load_from_disk(self.config.data_path)

        rough_names = ["rouge1","rouge2","rougeL","rougeLsum"]

        rough_metric = evaluate.load('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset = data_samsum_pt['test'][0:1],
            metric = rough_metric,
            model = model_pegasus,
            tokenizer = tokenizer,
            batch_size = 4,
            device = device,
            column_text = "dialogue",
            column_summary = "summary"
        )

        rough_dict = dict((rn,score[rn]) for rn in rough_names)

        df = pd.DataFrame(rough_dict,  index = ['pegasus'])
        df.to_csv(self.config.metric_file_name,index=False)



In [26]:
#gets add in pipeline part
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e